In [1]:
from ipyleaflet import Map, projections, LayersControl, LayerGroup
from ipyleaflet import ScaleControl, FullScreenControl, SplitMapControl

from localtileserver import examples
from localtileserver import TileClient, get_leaflet_tile_layer

import pystac

In [2]:
catalog = pystac.read_file("/home/jovyan/water-bodies/dp8i2kiv/catalog.json")

catalog.describe()

* <Catalog id=catalog>
  * <Item id=S2B_10TFK_20210713_0_L2A>
  * <Item id=S2A_10TFK_20210718_0_L2A>


In [3]:
m = None

index = 0
groups = []
for item in catalog.get_all_items():

    client = TileClient(item.get_assets()["data"].get_absolute_href())
    t = get_leaflet_tile_layer(client)

    if not m:
        m = Map(center=client.center(), zoom=client.default_zoom)

        control = LayersControl(position='topright')
        m.add_control(control)
    
    groups.append(LayerGroup(name=item.id, layers=()))
    
    m.add_layer(groups[index])
    groups[index].add_layer(t)
     
    index = index + 1 
    
m

Map(center=[40.15302581081341, -121.06648900517655], controls=(ZoomControl(options=['position', 'zoom_in_text'…